In [2]:
from fastai.vision import *
from pathlib import Path
import pickle
import sys
from operator import attrgetter
PATH = Path('data')
list(PATH.iterdir())

[PosixPath('data/challenge-2019-classes-description-500.csv'),
 PosixPath('data/fulltest'),
 PosixPath('data/challenge-2019-validation-detection-human-imagelabels.csv'),
 PosixPath('data/Xclassdescription.csv'),
 PosixPath('data/challenge-2019-validation-detection-bbox.csv'),
 PosixPath('data/fulltrain'),
 PosixPath('data/challenge-2019-train-detection-bbox.csv'),
 PosixPath('data/fullvalidation'),
 PosixPath('data/challenge-2019-train-detection-human-imagelabels.csv')]

In [6]:
class_names = pd.read_csv(PATH/'challenge-2019-classes-description-500.csv', header=None, names=["LabelName", "Class"])

#### Labels, bounding boxes for training

In [7]:
il_train = ImageList.from_folder(PATH/"fulltrain/", convert_mode='L') # contains 1743042 images
print(len(il_train))
bboxes_train = pd.read_csv(PATH/'challenge-2019-train-detection-bbox.csv')
labels_train = bboxes_train.merge(class_names, on="LabelName")
labels_train.loc[:, "ImageID"] = labels_train.loc[:, "ImageID"].astype('category')

1743042


In [8]:
#ids = labels_train.ImageID.unique()
ids_train = list(map(attrgetter('stem'), list(il_train.items)))
df_dict = labels_train.to_dict("list") 
x = {image_id : {"bboxes" : [], "Classes" : []} for image_id in ids_train}
[x.update({image_id: {"bboxes" : x[image_id]["bboxes"] + [[xmin, xmax, ymin, ymax]], "Classes" : x[image_id]["Classes"] + [c]}} ) 
     for c, image_id, xmin, xmax, ymin, ymax in zip(df_dict["Class"], df_dict["ImageID"], df_dict["XMin"], df_dict["XMax"], df_dict["YMin"], df_dict["YMax"])]
img2bbox_train = {image_id : [x[image_id]["bboxes"], x[image_id]["Classes"]] for image_id in ids_train}

#### Labels, bounding boxes for validation

In [9]:
il_val = ImageList.from_folder(PATH/"fullvalidation/", convert_mode='L') # contains 41620 images
bboxes_val = pd.read_csv(PATH/'challenge-2019-validation-detection-bbox.csv')
labels_val = bboxes_val.merge(class_names, on="LabelName")
labels_val.loc[:, "ImageID"] = labels_val.loc[:, "ImageID"].astype('category')

In [10]:
#ids = labels_val.ImageID.unique()
ids_val = list(map(attrgetter('stem'), list(il_val.items)))
df_dict = labels_val.to_dict("list") 
x = {image_id : {"bboxes" : [], "Classes" : []} for image_id in ids_val}
[x.update({image_id: {"bboxes" : x[image_id]["bboxes"] + [[xmin, xmax, ymin, ymax]], "Classes" : x[image_id]["Classes"] + [c]}} ) 
     for c, image_id, xmin, xmax, ymin, ymax in zip(df_dict["Class"], df_dict["ImageID"], df_dict["XMin"], df_dict["XMax"], df_dict["YMin"], df_dict["YMax"])]
img2bbox_val = {image_id : [x[image_id]["bboxes"], x[image_id]["Classes"]] for image_id in ids_val}

In [11]:
img2bbox = {**img2bbox_train, **img2bbox_val}

In [12]:
assert list(img2bbox_train.keys())==ids_train
assert list(img2bbox_val.keys())==ids_val
assert list(img2bbox.keys())==ids_train+ids_val

In [13]:
get_y_func = lambda o:img2bbox[o.stem]
#get_y_func = lambda o:print(o)

In [17]:
def get_data(bs, size):
    #src = ImageList.from_folder(PATH/'fulltrain', convert_mode='L')
    #src = src.split_by_files(il_val_sub)
    #src = ObjectItemList.from_folder(PATH/'fulltrain', convert_mode='L')
    src = ObjectItemList.from_folder(path=PATH, convert_mode='L')
    src = src.split_by_folder(train='fulltrain', valid='fullvalidation')
    #src = src.split_by_files(img2bbox_val.keys())
    src = src.label_from_func(get_y_func)
    return src.databunch(path=PATH, bs=bs, collate_fn=bb_pad_collate)

In [18]:
data = get_data(4, 128) 

You can deactivate this warning by passing `no_check=True`.


/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/fastai/basic_data.py:269: UserWarning: It's not possible to collate samples of your dataset together in a batch.
  warn(message)


In [20]:
with open(PATH/'databunch.pickle', 'wb') as f:
    pickle.dump(data, f)

In [19]:
data

ImageDataBunch;

Train: LabelList (1743042 items)
x: ObjectItemList
Image (1, 768, 1024),Image (1, 680, 1024),Image (1, 1024, 768),Image (1, 768, 1024),Image (1, 685, 1024)
y: ObjectCategoryList
ImageBBox (768, 1024),ImageBBox (680, 1024),ImageBBox (1024, 768),ImageBBox (768, 1024),ImageBBox (685, 1024)
Path: data;

Valid: LabelList (41620 items)
x: ObjectItemList
Image (1, 768, 1024),Image (1, 768, 1024),Image (1, 765, 1024),Image (1, 842, 1024),Image (1, 768, 768)
y: ObjectCategoryList
ImageBBox (768, 1024),ImageBBox (768, 1024),ImageBBox (765, 1024),ImageBBox (842, 1024),ImageBBox (768, 768)
Path: data;

Test: None